In [1]:
import pandas as pd
import xarray as xr


In [2]:
from pangeo_forge_recipes.patterns import ConcatDim, FilePattern
from pangeo_forge_recipes.recipes import XarrayZarrRecipe,setup_logging


In [3]:
dates = pd.date_range('2009-07-01', '2010-06-30', freq='D')

In [4]:
dates

DatetimeIndex(['2009-07-01', '2009-07-02', '2009-07-03', '2009-07-04',
               '2009-07-05', '2009-07-06', '2009-07-07', '2009-07-08',
               '2009-07-09', '2009-07-10',
               ...
               '2010-06-21', '2010-06-22', '2010-06-23', '2010-06-24',
               '2010-06-25', '2010-06-26', '2010-06-27', '2010-06-28',
               '2010-06-29', '2010-06-30'],
              dtype='datetime64[ns]', length=365, freq='D')

In [5]:
URL_FORMAT = (
    "https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/"
    "eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y{time:%Y}m{time:%m}d{time:%d}.1d_somxl010.nc"
)



In [6]:
def make_url(time):
    return URL_FORMAT.format(time=time)



In [7]:
truc=make_url(dates[0])

In [8]:
time_concat_dim = ConcatDim("time", dates, nitems_per_file=1)
time_concat_dim

ConcatDim(name='time', nitems_per_file=1)

In [9]:
pattern = FilePattern(make_url, time_concat_dim)
pattern

<FilePattern {'time': 365}>

In [10]:
for index, url in pattern.items():
    print(index)
    print(url)
    # Stop after the 3rd filepath (September 3rd, 1981)
    if 'y2009m07d05' in url:
        break

Index({DimIndex(name='time', index=0, sequence_len=365, operation=<CombineOp.CONCAT: 2>)})
https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc
Index({DimIndex(name='time', index=1, sequence_len=365, operation=<CombineOp.CONCAT: 2>)})
https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d02.1d_somxl010.nc
Index({DimIndex(name='time', index=2, sequence_len=365, operation=<CombineOp.CONCAT: 2>)})
https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d03.1d_somxl010.nc
Index({DimIndex(name='time', index=3, sequence_len=365, operation=<CombineOp.CONCAT: 2>)})
https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d04.1d_somxl010.nc
Index({DimIndex(

In [11]:
target_chunks = {"time_counter": 1, "y": 1500}
recipe = XarrayZarrRecipe(pattern, target_chunks=target_chunks)
recipe

XarrayZarrRecipe(file_pattern=<FilePattern {'time': 365}>, storage_config=StorageConfig(target=FSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7ff898bc61a0>, root_path='/tmp/tmplrepsb7m/v72yGbFZ'), cache=CacheFSSpecTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7ff898bc61a0>, root_path='/tmp/tmplrepsb7m/wErHzYkz'), metadata=MetadataTarget(fs=<fsspec.implementations.local.LocalFileSystem object at 0x7ff898bc61a0>, root_path='/tmp/tmplrepsb7m/bxUs5NeM')), inputs_per_chunk=1, target_chunks={'time_counter': 1, 'y': 1500}, cache_inputs=True, copy_input_to_local_file=False, consolidate_zarr=True, consolidate_dimension_coordinates=True, xarray_open_kwargs={}, xarray_concat_kwargs={}, delete_input_encoding=True, process_input=None, process_chunk=None, lock_timeout=None, subset_inputs={}, open_input_with_kerchunk=False)

In [12]:
setup_logging()

In [13]:
recipe.file_pattern

<FilePattern {'time': 365}>

In [14]:
run_function = recipe.to_function()

In [ ]:
run_function()

pangeo_forge_recipes.recipes.xarray_zarr - INFO - Caching input 'Index({DimIndex(name='time', index=0, sequence_len=365, operation=<CombineOp.CONCAT: 2>)})'
pangeo_forge_recipes.storage - INFO - Caching file 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc'
pangeo_forge_recipes.storage - INFO - Copying remote file 'https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/eNATL60/eNATL60-BLBT02/1d/eNATL60-BLBT02_y2009m07d01.1d_somxl010.nc' to cache


In [ ]:
mld_zarr_prune = xr.open_zarr(recipe.target_mapper, consolidated=True)
mld_zarr_prune